In [ ]:
%run sources/slicer.py
%run sources/time_series_to_intervals.py
%run sources/time_series_event_merge.py

main_value = 1 #variable
main_feature = "patient"


In [ ]:
feature = "distance"
p = pd.read_csv(f"processed_data/diabetes/diabetes_all_clean.csv")
set(p["label"])

In [ ]:
time_feature = "vt"

timeseries_df_list = {"paths": [],
                       "group_features": [],
                       "bucket_values": []}

event_df_list = {"paths": ["processed_data/diabetes/diabetes_all_clean.csv"],
                  "group_features": ["label"],
                  "value_feature": ["value"]}

slicer_params = {"stride": 12, "ow": 12, "ww": 4, "pw": 12, "granularity": "hours"} #variable

final_odf, final_pdf = samples_from_dataset(main_value, main_feature, time_feature, timeseries_df_list, event_df_list, slicer_params)

In [ ]:
final_pdf['label'][4]

In [ ]:
def hypoglicemia_from_prediction_window(dfs):
    r, ids =[], []
    for i, df in enumerate(dfs["label"]):
        ids.append(i)
        if len(df) > 0 and "Hypoglycemic symptoms" in set(df["label"]):
            r.append(True) 
        else:
            r.append(False)

    return pd.DataFrame({'Hypoglycemia':r, 'i':ids})




In [ ]:
def df_to_sequences(df):
    df["sequence"] = df.apply(lambda row: (row['vt'], row['label'], row['value']), axis=1)
    r =df.groupby("i")["sequence"].agg(list).reset_index()
    for i in range(len(r)):
        r.iloc[i]["sequence"].sort(key=lambda x: x[0])
    return r

Z = df_to_sequences(final_odf)

In [ ]:
CLASS_LABEL = "Hypoglycemic symptoms"

In [ ]:
Z = Z[Z.apply(lambda x: CLASS_LABEL not in [y[1] for y in x.sequence], axis=1)]

In [ ]:
Z = pd.merge(Z, hypoglicemia_from_prediction_window(final_pdf), on='i', how='inner')

In [ ]:
Z.Hypoglycemia.mean()

In [ ]:
Z["class"] = Z["Hypoglycemia"]

In [ ]:
import json
from datetime import datetime

In [ ]:
ZPARAMS = {
    "main_value":main_value, 
    "main_feature":main_feature, 
    "time_feature":time_feature, 
    "timeseries_df_list":timeseries_df_list, 
    "event_df_list":event_df_list, 
    "slicer_params":slicer_params,
    "class": "Hypoglycemic symptoms"
}
TIME = datetime.now().strftime('%Y%m%d_%H%M')
ZNAME = f"{TIME}_diabetes_{main_value}_removed"


In [ ]:
Z[["sequence", "class"]].to_csv(f"datasets/{ZNAME}.csv", index=False)
with open(f"datasets/{ZNAME}.json", 'w') as json_file:
    json.dump(ZPARAMS, json_file, indent=2)
